In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents (9).json


{'intents.json': b'{\n\t"intents": [{\n\t\t\t"tag": "greeting",\n\t\t\t"patterns": [\n\t\t\t\t"Hi",\n\t\t\t\t"Hi Bot",\n\t\t\t\t"Hola",\n\t\t\t\t"Hi, how are you",\n\t\t\t\t"How are you",\n\t\t\t\t"Is anyone there?",\n\t\t\t\t"Hello",\n\t\t\t\t"Hey",\n\t\t\t\t"What\'s up",\n\t\t\t\t"Hi there",\n\t\t\t\t"Hey there",\n\t\t\t\t"Good morning",\n\t\t\t\t"Good afternoon",\n\t\t\t\t"Good evening",\n\t\t\t\t"Howdy",\n\t\t\t\t"Hello there"\n\t\t\t],\n\t\t\t"responses": [\n\t\t\t\t"What can i help you with today?",\n\t\t\t\t"How can I help you today?"\n\t\t\t],\n\t\t\t"context_set": ""\n\t\t},\n\t\t{\n\t\t\t"tag": "goodbye",\n\t\t\t"patterns": [\n\t\t\t\t"Bye",\n\t\t\t\t"See you later",\n\t\t\t\t"Goodbye",\n\t\t\t\t"Farewell",\n\t\t\t\t"Take care",\n\t\t\t\t"Catch you later",\n\t\t\t\t"Goodbye for now",\n\t\t\t\t"Bye for now",\n\t\t\t\t"Talk to you later",\n\t\t\t\t"See you soon",\n\t\t\t\t"Take it easy",\n\t\t\t\t"So long",\n\t\t\t\t"Adios",\n\t\t\t\t"Good night",\n\t\t\t\t"Later",\n\t\t\t\t"Ha

In [ ]:
import gensim.downloader as api
# Load pre-trained word2vec model
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import json
import random
import numpy as np
from gensim.models import KeyedVectors  # Import the Word2Vec model from Gensim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from tensorflow import keras
nltk.download('punkt')
nltk.download('stopwords')

# Load the Word2Vec model
word2vec_model = KeyedVectors.load_word2vec_format('path_to_word2vec_model', binary=True)

# Load the intents data
with open("intents.json", "r") as f:
    data = json.load(f)

# Preprocessing
vocab = set()
classes = []
documents = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern = pattern.rstrip("?")
        tokens = word_tokenize(pattern)
        tokens = [token.lower() for token in tokens if token not in set(stopwords.words("english"))]
        vocab.update(tokens)
        documents.append((tokens, intent["tag"]))
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Generate word embeddings for each token
X = []
y = []
for doc in documents:
    vecs = []
    for token in doc[0]:
        if token in word2vec_model:
            vecs.append(word2vec_model[token])  # Get the word embedding for the token
    if vecs:
        vecs = np.mean(vecs, axis=0)  # Calculate the mean word embedding for the document
        X.append(vecs)
        y.append(classes.index(doc[1]))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


JSONDecodeError: ignored

In [ ]:
# Build the model
model1 = Sequential()
model1.add(Dense(128, input_dim=len(X_train[0]), activation="relu"))  # Input layer with ReLU activation
model1.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model1.add(Dense(64, activation="relu"))  # Hidden layer with ReLU activation
model1.add(Dense(len(classes), activation="softmax"))  # Output layer with softmax activation for multi-class classification

model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])  # Compile the model

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto', restore_best_weights=True)
# Define an early stopping callback to monitor the validation loss during training
# It will stop training if the validation loss doesn't improve for 20 epochs
# verbose=1 prints messages about the early stopping process
# mode='auto' determines the direction of improvement (minimizing loss)
# restore_best_weights=True restores the weights of the model to the ones with the best performance on the validation set

# Train the model
model1.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=5, callbacks=early_stop)
# Train the model using the training data
# Perform 200 epochs, use 20% of the training data as validation set
# Batch size is 5, which means the model will be updated after every 5 samples
# The early_stop callback will monitor the validation loss and stop training if it doesn't improve

# Evaluate the model
loss, accuracy = model1.evaluate(X_test, y_test)
# Evaluate the trained model using the testing data
# Calculate the loss and accuracy of the model on the testing data

print(f"Loss: {loss}, Accuracy: {accuracy}")
# Print the loss and accuracy of the model on the testing data


In [ ]:
def vectorize_sentence(sentence, model):
    sentence = sentence.rstrip("?")  # Remove trailing question marks
    # Tokenize the sentence into words
    sentence_words = nltk.word_tokenize(sentence)
    # Lemmatize each word and remove stopwords
    sentence_words = [lemmatizer.lemmatize(word.lower(), pos='v') for word in sentence_words if word not in set(stopwords.words("english"))]
    # Create a vector representation
    vecs = []
    for word in sentence_words:
        if word in model.key_to_index:
            vecs.append(model[word])  # Get the word embedding for the word
    if vecs:
        vecs = np.mean(vecs, axis=0)  # Calculate the mean vector representation for the sentence
    else:
        vecs = np.zeros(model.vector_size)  # Use zero vector if no word embeddings are found
    return vecs

print("Hi there, this is Genie. I'm so happy to chat with you today! 😊")
print("Could you please be more specific with your questions.")
print("I'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):")

while True:
    inp = input("You: ")  # Get user input
    if inp.lower() == "quit":
        # If the user enters "quit", find the "goodbye" intent and print a random response
        for intent in data["intents"]:
            if intent["tag"] == "goodbye":
                print("Bot: " + random.choice(intent["responses"]))
        break  # Exit the loop and end the conversation

    vec = vectorize_sentence(inp, model)  # Vectorize the user input using the pre-trained word embeddings
    results = model1.predict(np.array([vec]))  # Make a prediction using the trained model
    results_index = np.argmax(results)  # Get the index of the predicted intent
    tag = classes[results_index]

    if results[0][results_index] > 0.5:
        # If the confidence score is above 0.5, find the intent and print a response
        for intent in data["intents"]:
            if intent["tag"] == tag:
                if intent["tag"] == "goodbye" and inp == 'quit':
                    # If the intent is "goodbye" and the user entered "quit", print a goodbye message and end the conversation
                    print("Bot: " + random.choice(intent["responses"]))
                    print("Bot: Goodbye!")
                    break
                if intent["tag"] == "goodbye" and inp != 'quit':
                    # If the intent is "goodbye" but the user entered something other than "quit", print a specific message
                    print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist.")
                else:
                    # Print a random response from the intent
                    print("Bot: " + random.choice(intent["responses"]))
                    break
    else:
        # If the confidence score is below 0.5, print a generic message
        print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist.")


In [ ]:
path='/content/drive/My Drive/Colab Notebooks' #Declare the path where the code and model file has been placed

In [ ]:
model1.save('Genie_word2vec.h5') #save the model